In [36]:
import numpy as np
import pandas as pd
import os

In [40]:
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"] 


In [94]:
#PARTICIPANT FRAMES

def part_frames(json_file):
    
    data = pd.DataFrame(json_file["info"]["frames"][15]["participantFrames"])
    
    data = data.transpose().drop(columns=["currentGold", "goldPerSecond", "participantId", "position", "timeEnemySpentControlled", "level"])
    data.reset_index(drop=True, inplace=True)

    damage_stats = pd.json_normalize(data["damageStats"])
    champion_stats = pd.json_normalize(data["championStats"])

    pf_all = pd.concat([data, damage_stats, champion_stats], axis=1).drop(columns=["championStats","damageStats"])
    pf_all = pf_all.astype(int)

    conditions = [
            (pf_all.index >= 0) & (pf_all.index <= 4),
            (pf_all.index >= 5) & (pf_all.index <= 9)
        ]
    values = [100, 200]

    pf_all['team'] = np.select(conditions, values, default=0)

    pf_all = pf_all.groupby("team").sum().reset_index()

    return pf_all


#EVENTS & TARGET

def find_indices(data):
    monster_indices = []
    for index, item in enumerate(data):
        if item.get('type') in look_events:
            monster_indices.append(index)
    return monster_indices

def get_events(json_file):
    elements_filtered = []
    for i in range(16):
        try:
            events = json_file["info"]["frames"][i]["events"]
        except IndexError:
            break
        monster_indices = find_indices(events)
        if monster_indices:
            new_elements = [events[e] for e in monster_indices]
            elements_filtered.extend(new_elements)

    events_filtered = pd.DataFrame(elements_filtered)
    return events_filtered


def transform_events(json_file):
    required_columns = ["killerId", "type", "killType", "monsterSubType", "monsterType", "towerType"]

    events = get_events(json_file)

    missing_columns = [col for col in required_columns if col not in events.columns]

    for column in missing_columns:
        events[column] = np.nan

    events = events[required_columns]
    events = events[events["killerId"] != 0]

    events.loc[events["monsterType"] == "DRAGON", "monsterType"] = events.loc[events["monsterType"] == "DRAGON", "monsterSubType"]

    events = events.drop(columns=["monsterSubType"])

    events["kills"] = (events["type"] == "CHAMPION_KILL").astype(int)

    one_hot_encoded = pd.get_dummies(events[['killType', "monsterType", "towerType"]])

    events_encoded = pd.concat([events[['killerId', "kills"]], one_hot_encoded], axis=1)
    
    if 'killType' in events_encoded.columns:
        events_encoded.drop(columns=['killType'], inplace=True)
    if 'monsterType' in events_encoded.columns:
        events_encoded.drop(columns=['monsterType'], inplace=True)
    if 'towerType' in events_encoded.columns:
        events_encoded.drop(columns=['towerType'], inplace=True)
    
    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    events_encoded["killerId"] = events_encoded["killerId"] - 1

    events_encoded = events_encoded.groupby("killerId").sum().reset_index()

    conditions = [
        (events_encoded['killerId'] >= 0) & (events_encoded['killerId'] <= 4),
        (events_encoded['killerId'] >= 5) & (events_encoded['killerId'] <= 9)
    ]
    values = [100, 200]

    events_encoded['team'] = np.select(conditions, values, default=0)

    events_encoded = events_encoded.groupby("team").sum().reset_index().drop(columns=["killerId"])
    
    return events_encoded


def merge_dfs(json_file):
    events = transform_events(json_file)
    frames = part_frames(json_file)
    
    dfs = frames.merge(events,how="left",on="team")
    
    dfs["matchId"] = json_file["metadata"]["matchId"]
    
    last_event = json_file["info"]["frames"][-1]["events"][-1]
    dfs["target"] = dfs["team"].apply(lambda x: 1 if x == last_event.get("winningTeam") else 0)
    
    dfs.drop(columns=["team"],inplace=True)
    
    return dfs



In [95]:

folder_path = "data/IRON-Timeline" #Change to your folder path
all_events = None


for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        json_file = pd.read_json(file_path)
           
        try:             
            if all_events is None:
                all_events = merge_dfs(json_file)
            else:
                all_events = pd.concat([all_events, merge_dfs(json_file)], ignore_index=True)
        
        except IndexError:
            continue
     
columns_to_convert = all_events.columns[~all_events.columns.isin(['matchId',"target"])]   
    
all_events[columns_to_convert] = all_events[columns_to_convert].fillna(0).astype(int) 

In [96]:
#DIFERENCIAS ENTRE PARTIDAS


# Seleccionar todas las columnas excepto "matchId y"
columnas_no_matchId = all_events.columns[~ all_events.columns.isin(["matchId", "target"])]

# Seleccionar solo las columnas numéricas
columnas_numericas = all_events.select_dtypes(include=[np.number]).drop(columns = "target")

# Crear un nuevo DataFrame vacío con la misma columna "matchId, target" y las columnas numéricas
all_events_diff = pd.DataFrame(columns=["matchId", "target"] + list(columnas_numericas.columns))

# Obtener el número total de filas en df_trabajo
total_filas = columnas_numericas.shape[0]

# Iterar a través de las filas de columnas_numericas
for i in range(0, total_filas - 1, 2):
    # Obtener las dos filas a restar
    fila1 = columnas_numericas.iloc[i]
    fila2 = columnas_numericas.iloc[i+1]
    
    # Calcular la resta de las dos filas
    resta_filas = fila1 - fila2
    
    # Agregar la resta al nuevo DataFrame df_partida_diff
    all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)

# Asignar el primer valor de las columnas booleanas en cada fila del nuevo DataFrame
for columna in all_events.select_dtypes(include=[bool]):
    all_events_diff[columna] = all_events[columna].iloc[::2].reset_index(drop=True)

# Asignar el primer valor de la columna "matchId" en cada fila del nuevo DataFrame
all_events_diff["matchId"] = all_events["matchId"].iloc[::2].reset_index(drop=True)

# Asignar el primer valor de la columna "target" en cada fila del nuevo DataFrame
all_events_diff["target"] = all_events["target"].iloc[::2].reset_index(drop=True)

/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7250/3593382592.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)
/tmp/ipykernel_7

In [104]:
pd.set_option('display.max_columns', None)

all_events_diff

,matchId,target,jungleMinionsKilled,minionsKilled,totalGold,xp,magicDamageDone,magicDamageDoneToChampions,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,bonusMagicPenPercent,ccReduction,cooldownReduction,health,healthMax,healthRegen,lifesteal,magicPen,magicPenPercent,magicResist,movementSpeed,omnivamp,physicalVamp,power,powerMax,powerRegen,spellVamp,kills,killType_KILL_FIRST_BLOOD,killType_KILL_MULTI,monsterType_CHEMTECH_DRAGON,monsterType_FIRE_DRAGON,monsterType_RIFTHERALD,towerType_OUTER_TURRET,monsterType_WATER_DRAGON,monsterType_HEXTECH_DRAGON,monsterType_AIR_DRAGON,monsterType_EARTH_DRAGON,towerType_INNER_TURRET,killType_KILL_ACE,towerType_BASE_TURRET,towerType_NEXUS_TURRET
0,LA2_1302593691,0,-18,-28,-10498,-1398,20946,183,-1189,-101528,-12708,13345,-43110,-15219,14849,37472,-2693,2692,0,173,16,0,0,-234,-152,0,0,-25,0,153,386,-87,-9,13,0,10,-52,-2,0,1242,172,-98,0,-14,-1,0,-1,1,1,-1,0,0,0,0,0,0,0,0
1,LA2_1216779473,0,50,-122,-3927,-2769,42767,4011,-3530,-72832,-10754,8307,-24172,-6669,4707,5891,70,-70,0,66,57,0,0,-226,-50,0,0,-36,0,223,231,84,0,0,7,-31,-78,-12,0,434,495,-262,0,-4,1,0,0,0,0,-1,1,0,0,0,0,0,0,0
2,LA2_1284938323,0,-8,-2,-2589,-1853,-30518,-8799,10680,32268,10683,-15131,4653,464,-3028,2901,-1422,1422,0,-80,67,0,0,8,-133,0,0,10,0,-431,-507,77,3,0,0,-21,-247,2,0,1172,692,73,0,-6,1,0,0,0,-1,0,0,-1,0,0,0,0,0,0
3,LA2_1287471535,0,16,14,-75,630,-9309,-5163,4390,31555,-878,-650,26715,-6660,4360,4470,-620,619,0,-35,-7,0,0,14,275,0,0,-11,0,-644,-17,46,6,-18,0,16,-124,-2,0,-221,-2135,-62,0,-3,-1,0,1,0,-1,0,-1,0,0,0,0,0,0,0
4,LA2_1195433386,1,12,-13,-467,-1120,-21851,1381,-1924,2448,-4739,5033,-18273,-2068,1819,1130,1290,-1289,0,157,-68,0,0,-108,59,0,0,-23,0,-1266,-1027,118,13,36,-7,-41,-172,-6,0,9612,9962,-87,0,-8,1,0,0,1,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4700,LA2_1283443111,0,-28,-89,-2889,-2247,-39163,334,-1333,2494,2714,-3465,-52161,490,-2238,-15488,-2558,2558,0,-42,262,0,0,-52,-45,0,0,11,0,-1041,-773,-9,-6,-12,0,51,190,-2,0,-348,-1135,210,0,0,1,0,0,0,-1,-1,0,0,0,1,0,0,0,0
4701,LA2_1307378414,0,9,92,2054,3487,44388,3683,-3875,1957,-5896,10821,48413,-4332,9065,2068,-2118,2118,0,-80,21,0,0,-34,116,0,0,5,0,-415,1000,-28,5,-18,10,13,-90,5,0,575,-653,85,0,1,-1,-1,0,-1,1,0,1,0,0,0,0,0,0,0
4702,LA2_1263285710,0,-20,-6,-4489,-3432,-34277,-7212,7402,4399,-305,-1388,-44121,-7881,6378,-14240,-363,362,0,31,-49,0,0,-74,186,0,0,-7,0,-262,-1111,-31,8,-24,0,34,28,0,0,-1116,-2512,-129,0,-4,1,-1,-1,0,0,-1,0,0,0,0,0,0,0,0
4703,LA2_1253741053,0,-20,29,-1385,-1197,-14360,-6190,5773,18758,3582,-2340,15056,-1879,2704,10661,729,-729,0,-7,-110,0,0,-29,188,0,0,-52,0,-1249,-122,-3,0,-30,-10,-40,-132,-8,0,-190,-1376,-168,0,2,1,1,0,-1,-1,0,0,0,-1,0,0,0,0,0
